In [ ]:
# Databricks notebook source
df =  spark.read.csv("/FileStore/tables/sales_data_analise_dados_hist.csv", header = True , inferSchema=True )
from pyspark.sql import functions as F

COMMAND ----------

In [ ]:
df.show()

COMMAND ----------

# verificando a avaliação média total, e as vendas totais, para sabermos se há correspondência entre as avaliações e as vendas
# agrupar por produto, fazer a média do rating ( avaliação ), e somar o total de vendas

In [ ]:
product_rating_sale = df.groupBy('product_name').agg(F.avg('rating').alias('average_rating'), F.sum('sales_amount').alias('total_sales'))

In [ ]:
product_rating_sale.orderBy(F.desc('average_rating')).show()

COMMAND ----------

# Verificar os produtos mais vendidos, porém que contém as avaliações mais baixas (abaixo da média)
# criando dataframe novo, e filtrando o resultado , utilizando o dataframe anterior, apenas pegando as avaliações menores que 3.

In [ ]:
low_rated_high_sales = product_rating_sale.filter(product_rating_sale.average_rating < 3).orderBy(F.desc('total_sales'))

In [ ]:
low_rated_high_sales.show()

COMMAND ----------

# verifica se as promoções impulsionam as vendas mesmo dos itens que estão com a avaliação mais baixa ( < 3)
# buscando a soma das vendas, dos produtos com menor avaliação ( abaixo de rating < 3),agrupados por nome do produto ( product_name )
# neste caso os filtros do data frame, foram realizados anes do agrupamento e agregação ( soma )

In [ ]:
low_rated_sales_on_promo = df.filter((df.rating <3) & (df.promotion == True)).groupBy('product_name').agg(F.sum('sales_amount').alias('sales_on_promo'))

In [ ]:
low_rated_sales_on_promo.orderBy(F.desc('sales_on_promo')).show()

COMMAND ----------

# buscar a quantidade de dias de vendas   agrupado por mês e ano  o total de dias ( mês e ano )

In [ ]:
total_days = df.groupBy(F.month('sale_date').alias('month'), F.year ('sale_date').alias("year")).agg(F.count('sale_date').alias ('total_days'))

In [ ]:
total_days.orderBy('year','month').show()

COMMAND ----------

# melhorar a consulta, buscando todos os produtos que estão com o rating baixo  , e se eles tiveram promoção associadas a ele
# mais uma vez PRIMEIRO FILTROS ( WHERE ), depois agrupamento

In [ ]:
total_days_promo = df.filter((df.rating < 3) & (df.promotion == True)).groupBy(F.month('sale_date').alias('month'), F.year ('sale_date').alias("year")).agg(F.count('sale_date').alias ('total_days_promo'))

In [ ]:
total_days_promo.orderBy ('year','month').show()

COMMAND ----------

JOIN, dos dataframes total_days_promo com total_days

In [ ]:
day_promo_non_promo = total_days_promo.join(total_days, on = ['month','year'])

In [ ]:
day_promo_non_promo.orderBy('year','month').show()

COMMAND ----------

# soma total das vendas dentro do mês, agrupados por mês e ano

In [ ]:
total_days_sales = df.groupBy(F.month('sale_date').alias('month'), F.year ('sale_date').alias("year")).agg(F.sum('sales_amount').alias ('total_sales'))
total_days_sales.orderBy('year','month').show()

COMMAND ----------

total de vendas dos  produtos com baixa avaliação e com promoção

In [ ]:
total_days_sales_promo = df.filter((df.rating < 2) & (df.promotion == True)).groupBy(F.month('sale_date').alias('month'), F.year ('sale_date').alias("year")).agg(F.sum('sales_amount').alias ('total_sales_promo'))

In [ ]:
total_days_sales_promo.orderBy('year','month').show()

COMMAND ----------

#fazer o join de vendas , encadeando todos os data frames anteriores?

In [ ]:
low_rate_sales = total_days_sales.join(total_days_sales_promo, on =['year','month']).join(total_days_promo, on = ['year','month']).join(total_days, on =['month','year'])           

In [ ]:
low_rate_sales.orderBy('year','month').show()

COMMAND ----------